## Logistic Regression

Breast Cancer data from [the UCI repository](http://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29) contains records corresponding to 
cases of observed tumors.   There are a number of observations for each and a categorisation in the `class` column: 2 for benign (good), 4 for malignant (bad).  Your task is to build a logistic regression model to classify these cases. 

The data is provided as a CSV file.  There are a small number of cases where no value is available, these are indicated in the data with `?`. I have used the `na_values` keyword for `read_csv` to have these interpreted as `NaN` (Not a Number).  Your first task is to decide what to do with these rows. You could just drop these rows or you could [impute them from the other data](http://scikit-learn.org/stable/modules/preprocessing.html#imputation-of-missing-values).

You then need to follow the procedure outlined in the lecture for generating a train/test set, building and evaluating a model. Your goal is to build the best model possible over this data.   Your first step should be to build a logistic regression model using all of the features that are available.
  

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score, mean_squared_error, precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_selection import RFE
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
bcancer = pd.read_csv("files/breast-cancer-wisconsin.csv", na_values="?")
bcancer.head()

,sample_code_number,clump_thickness,uniformity_cell_size,uniformity_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1.0,3,1,1,2
1,1002945,5,4,4,5,7,10.0,3,2,1,2
2,1015425,3,1,1,1,2,2.0,3,1,1,2
3,1016277,6,8,8,1,3,4.0,3,7,1,2
4,1017023,4,1,1,3,2,1.0,3,1,1,2


In [ ]:
bcancer.describe(include='all')

In [ ]:
imp = IterativeImputer(max_iter=10, verbose=0)
imp.fit(bcancer)
imputed_bcancer = imp.transform(bcancer)
imputed_bcancer = pd.DataFrame(imputed_bcancer, columns=bcancer.columns)

In [ ]:
imputed_bcancer.describe()

In [ ]:
train, test = train_test_split(imputed_bcancer, test_size=0.2)

,sample_code_number,clump_thickness,uniformity_cell_size,uniformity_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,class
440,608157.0,10.0,4.0,3.0,10.0,4.0,10.0,10.0,1.0,1.0,4.0
65,1116998.0,10.0,4.0,2.0,1.0,3.0,2.0,4.0,3.0,10.0,4.0
432,1277629.0,5.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0
545,1197527.0,5.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0
74,1126417.0,10.0,6.0,4.0,1.0,3.0,4.0,3.0,2.0,3.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...
498,1204558.0,4.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0
356,859164.0,5.0,3.0,3.0,1.0,3.0,3.0,3.0,3.0,3.0,4.0
645,1303489.0,3.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0
461,1268804.0,3.0,1.0,1.0,1.0,2.0,5.0,1.0,1.0,1.0,2.0


In [ ]:
X_train = train.drop(['class','sample_code_number'], axis=1)
y_train = train['class']
X_test = test.drop(['class','sample_code_number'], axis=1)
y_test = test['class']
logreg = linear_model.LogisticRegression(C=1e9)
logreg.fit(X_train, y_train)

LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

### Evaluation

To evaluate a classification model we want to look at how many cases were correctly classified and how many
were in error.  In this case we have two outcomes - benign and malignant.   SKlearn has some useful tools, the 
[accuracy_score]() function gives a score from 0-1 for the proportion correct.  The 
[confusion_matrix](http://scikit-learn.org/stable/modules/model_evaluation.html#confusion-matrix) function 
shows how many were classified correctly and what errors were made.  Use these to summarise the performance of 
your model (these functions have already been imported above).

In [ ]:
predicted = logreg.predict(X_train)
mse = mean_squared_error(y_train, predicted)
r2 = r2_score(y_train, predicted)
acc = accuracy_score(y_train,predicted)
cm1 = confusion_matrix(y_train,predicted)
model = pd.DataFrame([('Training',mse, r2, acc, cm1)], columns=['Data','MSE', 'R2 score','Accuracy','Confusion Matrix'])

predicted = logreg.predict(X_test)
mse = mean_squared_error(y_test, predicted)
r2 = r2_score(y_test, predicted)
acc = accuracy_score(y_test,predicted)
cm2 = confusion_matrix(y_test,predicted)
model.loc[1] = ['Testing', mse, r2, acc, cm2]

### Checkpoint

The checkpoint for this week workshop is to report accuracy on training and test set. Also, provide the confusion matrix to check for which class model is doing good and where error were are made. Based on these results, provide explaination about:
- Can we deploy this trained model in hospital's settings?
- Is model overfitting?
- For which class model is making error? Read about False Positive Rate (FPR) and False Negative Rate (FNR)

**On iLearn under Practical Week 6, make your submission in the form of results and their interpretation (1 paragraph) to get mark for this week checkpoint.**

In [ ]:
model

,Data,MSE,R2 score,Accuracy,Confusion Matrix
0,Training,0.157424,0.826324,0.960644,"[[355, 10], [12, 182]]"
1,Testing,0.057143,0.935941,0.985714,"[[91, 2], [0, 47]]"


The models have an accuracy of 0.96 and 0.98 when tested on training and testing data which is quite good for a logistic regression model, they can be deployed in a hospital setting. 

The R2 score for training set is very close to the testing set and therefore the model isn't overfitting. 

In [ ]:
tn = cm1[0, 0]
tp = cm1[1, 1]
fn = cm1[1, 0]
fp = cm1[0, 1]

model.loc[0,'FPR'] = fp / (fp + tn)
model.loc[0,'FNR'] = fn / (fn + tp)
model.loc[0,'TNR'] = tn / (tn + fp)
model.loc[0,'TPR'] = tp / (tp + fn)

tn = cm2[0, 0]
tp = cm2[1, 1]
fn = cm2[1, 0]
fp = cm2[0, 1]

model.loc[1,'FPR'] = fp / (fp + tn)
model.loc[1,'FNR'] = fn / (fn + tp)
model.loc[1,'TNR'] = tn / (tn + fp)
model.loc[1,'TPR'] = tp / (tp + fn)

In [ ]:
model

,Data,MSE,R2 score,Accuracy,Confusion Matrix,FPR,FNR,TNR,TPR
0,Training,0.157424,0.826324,0.960644,"[[355, 10], [12, 182]]",0.027397,0.061856,0.972603,0.938144
1,Testing,0.057143,0.935941,0.985714,"[[91, 2], [0, 47]]",0.021505,0.000000,0.978495,1.000000


#### Confusion Matrix

Total cell count: 699

#### Training data
Data count: 559

Benign cell count: 365

Malignant cell count: 194

97.26% benign cells and 93.81% malignant cells were correctly predicted. 

2.73% benign cells were predicted as being maignant and 6.18% maignant cells were predicted as benign.

#### Testing data
Data count: 140

Benign cell count: 93

Malignant cell count: 47

97.84% benign cells and 100% malignant cells were correctly predicted. 2.15% benign cells were predicted as being maignant

The testing model clearly better than testing data looking at the FP rate and FN rate.

### Feature Selection

Since you have many features available, one part of building the best model will be to select which features to use as input to the classifier. Your initial model used all of the features but it is possible that a better model can 
be built by leaving some of them out.   Test this by building a few models with subsets of the features - how do your models perform? 

This process can be automated.  The [sklearn RFE function](http://scikit-learn.org/stable/modules/feature_selection.html#recursive-feature-elimination) implements __Recursive Feature Estimation__ which removes 
features one by one, evaluating the model each time and selecting the best model for a target number of features.  Use RFE to select features for a model with 3, 4 and 5 features - can you build a model that is as good or better than your initial model?

In [ ]:
feature_cols = np.array(bcancer.columns)[1:-1]
X = imputed_bcancer.drop(['class','sample_code_number'], axis=1)
y = imputed_bcancer['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
cm = [None]*3
estimator = LogisticRegression(C=1e9)

for idx,i in enumerate([3,4,5]):
  selector = RFE(estimator, i)  # select 3 features for us
  selector = selector.fit(X, y)

  supp = selector.get_support()
  print("Selected features:", feature_cols[supp])
  print("Coeffs:", selector.estimator_.coef_)
  # test the model
  predicted = selector.predict(X)
  print("MSE:", mean_squared_error(y, predicted))
  print("R^2:", r2_score(y, predicted))
  print("Accuracy:",accuracy_score(y,predicted))
  cm[idx] = confusion_matrix(y,predicted)
  print("Classification matrix:",cm[idx])
  print("\n")

Selected features: ['uniformity_cell_shape' 'bland_chromatin' 'mitoses']
Coeffs: [[1.11182892 0.74475264 0.75385756]]
MSE: 0.23462088698140202
R^2: 0.7403558680171773
Accuracy: 0.9413447782546495
Classification matrix: [[442  16]
 [ 25 216]]


0 3
1 4
2 5
Selected features: ['clump_thickness' 'uniformity_cell_shape' 'bland_chromatin' 'mitoses']
Coeffs: [[0.55391074 0.84570179 0.69316168 0.72112276]]
MSE: 0.17167381974248927
R^2: 0.8100164887930565
Accuracy: 0.9570815450643777
Classification matrix: [[444  14]
 [ 16 225]]


0 3
1 4
2 5
Selected features: ['clump_thickness' 'uniformity_cell_shape' 'bare_nuclei' 'bland_chromatin'
 'mitoses']
Coeffs: [[0.49594084 0.5359611  0.44753375 0.47950766 0.5482071 ]]
MSE: 0.12589413447782546
R^2: 0.8606787584482415
Accuracy: 0.9685264663805436
Classification matrix: [[448  10]
 [ 12 229]]


0 3
1 4
2 5


## Conclusion

Write a brief conclusion to your experiment.  You might comment on the proportion of __false positive__ and __false negative__ classifications your model makes.  How useful would this model be in a clinical diagnostic setting? 

In [ ]:
for i in range(3):
  print("Feature count ",i+3)
  print("FPR: ",cm[i][0][1] / sum(cm[i][0]))
  print("FNR: ",cm[i][1][0] / sum(cm[i][1]))
  print("\n")

Feature count  3
FPR:  0.034934497816593885
FNR:  0.1037344398340249


Feature count  4
FPR:  0.03056768558951965
FNR:  0.06639004149377593


Feature count  5
FPR:  0.021834061135371178
FNR:  0.04979253112033195




The original test model still appears to be better than the feature selected models. From 3 to 5 feature selections, the error rates keep reducing. The accuracy and $R^2$ values appear to increase. 